In [1]:
import pandas as pd
import ipynb.fs.full.features as features_nltk

# df_train = pd.read_csv('./data/train.csv')
# df_test = pd.read_csv('./data/test.csv')

In [3]:
def concatenar_categoricos(df, columna1, columna2, N=0, df_aux=None, devolver_cols=False):
    '''Concatena dos columnas categoricas y hace one hot'''
    '''Ver referencia columna_a_ohe para los ultimos 3 parametros'''
    
    def limpiar_espacios_blanco(texto):
        try:
            return texto.replace(' ', '_')
        except:
            pass
        
    df = df.copy()
    df[columna1] = df[columna1].apply(limpiar_espacios_blanco)
    df[columna2] = df[columna2].apply(limpiar_espacios_blanco)
    df[columna1 + '_' + columna2] = df[columna1] + '_' +df[columna2]
    
    features_nltk
    return features_nltk.columna_a_ohe(df, columna1 + '_' + columna2, N, df_aux, devolver_cols)

In [ ]:
def cantidad_propiedad_misma_cantidad(df, agrupar_col, misma_cantidad_col):
    '''Cantidad de casas con la misma cantidad de banos que la actual'''
    '''Por ejemplo, retorna para cada propiedad, cuantas hay mas de su tipo con x banos ej.'''
    '''cantidad_propiedad_misma_cantidad(df, 'banos', 'tipodepropiedad'), para cada propiedad, se fija
    cuantas mas de su tipo con tantos banos hay'''
    '''Retorna el df con la columna agregada'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([agrupar_col, misma_cantidad_col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    def get_cantidad(col1, col2):
        cantidad = df_aux[(df_aux[agrupar_col] == col1) & (df_aux[misma_cantidad_col] == col2)]['cantidad']
        return cantidad.values[0] if len(cantidad.values > 0) else 0
        
    df['cantidad_' + agrupar_col + '_' + misma_cantidad_col] = df.apply(lambda x: get_cantidad(x[agrupar_col], x[misma_cantidad_col]), axis=1)
    return df

In [ ]:
def cantidad_atributo(df, col, df_test):
    '''Simil a value_counts. Dada una columna, agrega al df, cuantas propiedades existen con esa misma propiedad'''
    '''Se rellena en df_test, con los counts que se hicieron en df'''
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    prop_cantidad = {}

    def get_cantidad(col1):
        cantidad = df_aux[df_aux[col] == col1]['cantidad']
        if col1 in prop_cantidad:
            return prop_cantidad[col1]
        cantidad_retornar = cantidad.values[0] if len(cantidad.values > 0) else 0
        prop_cantidad[col1] = cantidad_retornar
        return cantidad_retornar
    
    df['cantidad_propiedades_en_' + col] = df.apply(lambda x: get_cantidad(x[col]), axis=1)
    
    df_test['cantidad_propiedades_en_' + col] = df_test.apply(
                            lambda x: prop_cantidad[x[col]] if x[col] in prop_cantidad else 0, axis=1)
    
    return df, df_test

In [ ]:
def ranking_atributo(df, col):
    '''Cuenta cuantas propiedades hay con los atributos de la columna recibida. Y para cada propiedad,
    se fija en que posicion esta del ranking. Por ejemplo, si hay dos propuedades, una con dos banos y otra 
    con uno solo, el rankin de las que tengan dos banos en 1, y de las propiedades que tengan un solo bano es 
    dos'''
    
    '''Rellena los nulos con "otros" '''
    df[col] = df[col].fillna('otros')
    
    df_aux = df.copy()
    df_aux = df_aux.groupby([col]).agg({'id': 'count'})
    df_aux = df_aux.rename(columns={'id':'cantidad'})
    df_aux = df_aux.reset_index()
    
    df_aux = df_aux.nlargest(df_aux.shape[0], 'cantidad') 
    lista_ranking = df_aux[col].tolist()
    rank = {}
    contador = 1
    
    for item in lista_ranking:
        rank[item] = contador
        contador = contador + 1

    def get_ranking(col1):
        item = df_aux[df_aux[col] == col1][col].values[0]
        return rank[item]
    
    df['ranking_en_' + col] = df.apply(lambda x: get_ranking(x[col]), axis=1)
    return df

In [ ]:
def aplicar_features(df_recibido):
    df = df_recibido.copy()    
    # -- 
    df = concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
    df = concatenar_categoricos(df, 'provincia', 'ciudad')
    # -- 
    display('Check-1.0')
    df = cantidad_atributo(df, 'provincia')
    display('Check-1.1')
    df = cantidad_atributo(df, 'ciudad')
    display('Check-1.2')
    df = cantidad_atributo(df, 'antiguedad')
    display('Check-1.3')
    df = cantidad_atributo(df, 'habitaciones')
    display('Check-1.4')
    df = cantidad_atributo(df, 'garages')
    display('Check-1.5')
    df = cantidad_atributo(df, 'banos')
    display('Check-1.6')
    df = cantidad_atributo(df, 'gimnasio')
    display('Check-1.7')
    df = cantidad_atributo(df, 'usosmultiples')
    display('Check-1.8')
    df = cantidad_atributo(df, 'escuelascercanas')
    display('Check-1.9')
    df = cantidad_atributo(df, 'centroscomercialescercanos')
    display('Check-1.10')
    df = cantidad_atributo(df, 'piscina')
    # --
    display('Check-2.0')
    df = ranking_atributo(df, 'provincia')
    display('Check-2.1')
    df = ranking_atributo(df, 'tipodepropiedad')
    display('Check-2.2')
    df = ranking_atributo(df, 'ciudad')
    display('Check-2.3')
    df = ranking_atributo(df, 'antiguedad')
    display('Check-2.4')
    df = ranking_atributo(df, 'habitaciones')
    display('Check-2.5')
    df = ranking_atributo(df, 'garages')
    display('Check-2.6')
    df = ranking_atributo(df, 'banos')
    display('Check-2.7')
    df = ranking_atributo(df, 'gimnasio')
    display('Check-2.8')
    df = ranking_atributo(df, 'usosmultiples')
    display('Check-2.9')
    df = ranking_atributo(df, 'escuelascercanas')
    display('Check-2.10')
    df = ranking_atributo(df, 'centroscomercialescercanos')    
    # --
    display('Check-3.0')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
    display('Check-3.1')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'ciudad')
    display('Check-3.2')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'antiguedad')
    display('Check-3.3')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'habitaciones')
    display('Check-3.4')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'garages')
    display('Check-3.5')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'banos')
    display('Check-3.6')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'gimnasio')
    display('Check-3.7')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'usosmultiples')
    display('Check-3.8')
    df = cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'piscina')
    display('Check-4.0')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'antiguedad')
    display('Check-4.1')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'habitaciones')
    display('Check-4.2')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'garages')
    display('Check-4.3')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'banos')
    display('Check-4.4')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'gimnasio')
    display('Check-4.5')
    df = cantidad_propiedad_misma_cantidad(df, 'ciudad', 'usosmultiples')
    display('Check-5.6')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'antiguedad')
    display('Check-5.7')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'habitaciones')
    display('Check-5.8')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'garages')
    display('Check-5.9')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'banos')
    display('Check-5.10')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'gimnasio')
    display('Check-5.11')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'usosmultiples')
    display('Check-5.12')
    df = cantidad_propiedad_misma_cantidad(df, 'provincia', 'piscina')
    display('Check-6.0')
    return df

In [ ]:
df = df_train
# df = df_test

display(df.columns)
display(df.shape)

#Ejs:
# concatenar_categoricos(df, 'tipodepropiedad', 'provincia')
# cantidad_propiedad_misma_cantidad(df, 'tipodepropiedad', 'provincia')
# cantidad_atributo(df, 'provincia')
# ranking_atributo(df, 'provincia')

df = aplicar_features(df)
# df.to_csv('masfeatures_train.csv', index=False)
# df.to_csv('masfeatures_test.csv', index=False)

In [ ]:
df_train = pd.read_csv('./masfeatures_train.csv')
df_eval = pd.read_csv('./masfeatures_test.csv')


# Correccion csv - strings que deberian ser floats
df_train['antiguedad'] = df_train['antiguedad'].apply(lambda antiguedad: None if antiguedad == 'otros' else antiguedad)
df_eval['antiguedad'] = df_eval['antiguedad'].apply(lambda antiguedad: None if antiguedad == 'otros' else antiguedad)
df_eval['antiguedad'] = df_eval['antiguedad'].astype(float)
df_train['antiguedad'] = df_train['antiguedad'].astype(float)

df_train['habitaciones'] = df_train['habitaciones'].apply(lambda x: None if x == 'otros' else x)
df_eval['habitaciones'] = df_eval['habitaciones'].apply(lambda x: None if x == 'otros' else x)
df_eval['habitaciones'] = df_eval['habitaciones'].astype(float)
df_train['habitaciones'] = df_train['habitaciones'].astype(float)

df_train['garages'] = df_train['garages'].apply(lambda x: None if x == 'otros' else x)
df_eval['garages'] = df_eval['garages'].apply(lambda x: None if x == 'otros' else x)
df_eval['garages'] = df_eval['garages'].astype(float)
df_train['garages'] = df_train['garages'].astype(float)

df_train['banos'] = df_train['banos'].apply(lambda x: None if x == 'otros' else x)
df_eval['banos'] = df_eval['banos'].apply(lambda x: None if x == 'otros' else x)
df_eval['banos'] = df_eval['banos'].astype(float)
df_train['banos'] = df_train['banos'].astype(float)

# Las columnas de tipo object
x = df_train.columns.to_series().groupby(df_train.dtypes).groups
display("Las columnas de tipo Object:")
display(list(x[list(x.keys())[len(list(x.keys()))-1]]))

df_eval.to_csv('masfeatures_train_f.csv', index=False)
df_train.to_csv('masfeatures_test_f.csv', index=False)

### Las mismas features anteriores, sin ohe - train

In [15]:
df_train = pd.read_csv('./data/train.csv')
df1, features1 = concatenar_categoricos(df_train, 'tipodepropiedad', 'provincia', 0, None, True)
df2, features2 = concatenar_categoricos(df_train, 'provincia', 'ciudad', 0, None, True)

features_borarr = features1 + features2
df_features_sin_ohe = pd.read_csv('./masfeatures_train_f.csv')
df_features_sin_ohe = df_features_sin_ohe.drop(features_borarr, axis=1)

display(len(df_features_sin_ohe.columns))
display(df_features_sin_ohe.columns)
display(df_features_sin_ohe)

df_features_sin_ohe.to_csv('masfeatures_train_f_no_ohe.csv', index=False)

69

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'tipodepropiedad_provincia',
       'provincia_ciudad', 'cantidad_propiedades_en_provincia',
       'cantidad_propiedades_en_ciudad', 'cantidad_propiedades_en_antiguedad',
       'cantidad_propiedades_en_habitaciones',
       'cantidad_propiedades_en_garages', 'cantidad_propiedades_en_banos',
       'cantidad_propiedades_en_gimnasio',
       'cantidad_propiedades_en_usosmultiples',
       'cantidad_propiedades_en_escuelascercanas',
       'cantidad_propiedades_en_centroscomercialescercanos',
       'cantidad_propiedades_en_piscina', 'ranking_en_provincia',
       'ranking_en_tipodepropiedad', 'ranking_en_ciudad',
       'ranking_en_antiguedad', 'rankin

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,cantidad_ciudad_banos,cantidad_ciudad_gimnasio,cantidad_ciudad_usosmultiples,cantidad_provincia_antiguedad,cantidad_provincia_habitaciones,cantidad_provincia_garages,cantidad_provincia_banos,cantidad_provincia_gimnasio,cantidad_provincia_usosmultiples,cantidad_provincia_piscina
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito_Juárez,Distrito_Federal,NaN,2.0,1.0,...,6127,10391,10392,7041,21062,19214,23514,54380,54746,56163
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa_en_condominio,AV. MEXICO,La_Magdalena_Contreras,Distrito_Federal,10.0,3.0,2.0,...,607,1842,1812,8904,23479,18587,23514,54380,54746,56163
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,257,783,778,4256,10492,9180,7527,19894,19431,18756
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo._de_México,1.0,2.0,1.0,...,46,149,148,1449,9406,11689,12013,38541,38547,39277
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,1704,9634,9306,2749,4188,5123,4434,19894,19431,18756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo._de_México,0.0,2.0,2.0,...,46,149,148,5649,9406,14322,12013,38541,38547,39277
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo._de_México,0.0,3.0,3.0,...,227,1932,1947,5649,22200,5976,9335,38541,38547,39277
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito_Juárez,Distrito_Federal,20.0,2.0,1.0,...,6127,10391,10392,9323,21062,19214,23514,54380,54746,56163
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito_Federal,20.0,4.0,0.0,...,99,50,3028,9323,5065,8472,2926,4410,54746,56163


### Las mismas features anteriores, sin ohe - train

In [16]:
df_test = pd.read_csv('./data/test.csv')
df1, features1 = concatenar_categoricos(df_test, 'tipodepropiedad', 'provincia', 0, None, True)
df2, features2 = concatenar_categoricos(df_test, 'provincia', 'ciudad', 0, None, True)

features_borarr = features1 + features2
df_features_sin_ohe = pd.read_csv('./masfeatures_test_f.csv')
df_features_sin_ohe = df_features_sin_ohe.drop(features_borarr, axis=1)

display(len(df_features_sin_ohe.columns))
display(df_features_sin_ohe.columns)
display(df_features_sin_ohe)

df_features_sin_ohe.to_csv('masfeatures_test_f_no_ohe.csv', index=False)

68

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'tipodepropiedad_provincia',
       'provincia_ciudad', 'cantidad_propiedades_en_provincia',
       'cantidad_propiedades_en_ciudad', 'cantidad_propiedades_en_antiguedad',
       'cantidad_propiedades_en_habitaciones',
       'cantidad_propiedades_en_garages', 'cantidad_propiedades_en_banos',
       'cantidad_propiedades_en_gimnasio',
       'cantidad_propiedades_en_usosmultiples',
       'cantidad_propiedades_en_escuelascercanas',
       'cantidad_propiedades_en_centroscomercialescercanos',
       'cantidad_propiedades_en_piscina', 'ranking_en_provincia',
       'ranking_en_tipodepropiedad', 'ranking_en_ciudad',
       'ranking_en_antiguedad', 'ranking_en_habit

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,cantidad_ciudad_banos,cantidad_ciudad_gimnasio,cantidad_ciudad_usosmultiples,cantidad_provincia_antiguedad,cantidad_provincia_habitaciones,cantidad_provincia_garages,cantidad_provincia_banos,cantidad_provincia_gimnasio,cantidad_provincia_usosmultiples,cantidad_provincia_piscina
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel_Hidalgo,Distrito_Federal,29.0,3.0,NaN,...,54,1213,1255,31,5828,1643,710,13668,13749,14136
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,252,1677,1798,670,47,123,278,1854,1980,1575
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito_Federal,0.0,2.0,1.0,...,503,1249,1249,1999,5263,4724,5881,13668,13749,14136
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco_de_Juárez,Guerrero,2.0,2.0,2.0,...,230,561,572,26,234,235,251,604,622,310
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo._de_México,10.0,2.0,1.0,...,153,231,232,1958,2219,2887,2894,9454,9435,9632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,75094,oportunidad!! se vende amplia casa en col. moc...,oportunidad!! ideal para oficina o casa habita...,Casa,Oriente 172 # 265,Venustiano_Carranza,Distrito_Federal,20.0,4.0,3.0,...,30,393,395,2287,1273,1500,2516,13668,13749,14136
59996,171847,colinas de ecatepec,"casa, sala comedor, patio de servicio, buenas ...",Casa,colinas,Ecatepec_de_Morelos,Edo._de_México,10.0,3.0,1.0,...,172,551,551,1958,5458,2887,3466,9454,9435,9632
59997,138313,estrene hermosa casa en sierra morena,hermosa casa lista para habitarse ubicada en f...,Casa,s/calle,Guadalupe,Nuevo_León,5.0,3.0,2.0,...,180,459,487,585,2136,1546,1212,3698,3869,3772
59998,271268,zen house i venta de linda casa con acabados ...,hermosa casa con acabados de lujo en fracciona...,Casa,Zen House l,Querétaro,Querétaro,0.0,2.0,1.0,...,1219,3032,3011,1452,568,509,1733,4061,4015,3766
